In [4]:
import pandas as pd 
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
df = pd.read_parquet(r'C:\Users\mauri\OneDrive\Escritorio\MLops\data\processed\modelo_dataset.parquet')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28894 entries, 0 to 28893
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      28894 non-null  object
 1   predictor  28894 non-null  object
dtypes: object(2)
memory usage: 451.6+ KB


In [10]:
# Crear el vectorizador TF-IDF
#vectorizer = TfidfVectorizer(min_df=5, max_df=0.85, ngram_range=(1, 2), max_features=30000, dtype=np.float32) 

# Transformar la columna 'overview' en una matriz TF-IDF
#matriz = vectorizer.fit_transform(df['predictor'])

# Reductir la dimensionalidad con SVD.
#svd = TruncatedSVD(n_components=100, random_state=42)

#matriz_reducida = svd.fit_transform(matriz)
#num_features = len(vectorizer.vocabulary_)

#import psutil
#import os

#def get_memory_usage():
#    process = psutil.Process(os.getpid())
#    mem_info = process.memory_info()
#    return mem_info.rss / (1024 * 1024)  # Convertir de bytes a MB

#print(f"Uso de memoria: {get_memory_usage()} MB")
#print(f"El vectorizador está utilizando {num_features} características.")
#print(f"Dimensiones reducidas a {matriz_reducida.shape[1]} componentes.")

El vectorizador está utilizando 30000 características.
Dimensiones reducidas a 100 componentes.
Uso de memoria: 532.01171875 MB


In [11]:
# Calcular la matriz de similitud del coseno
#cosine_sim = cosine_similarity(matriz_reducida, matriz_reducida)


#import psutil
#import os
#def get_memory_usage():
#    process = psutil.Process(os.getpid())
#    mem_info = process.memory_info()
#    return mem_info.rss / (1024 * 1024)  # Convertir de bytes a MB

#print(f"Uso de memoria: {get_memory_usage()} MB")

Uso de memoria: 2143.859375 MB


In [8]:
#def recommendacion(title, cosine_sim=cosine_sim):
    # Encuentra el índice de la película
#    idx = df.index[df['title'] == title].tolist()
#    if not idx:
#        return "Película no encontrada"
#    idx = idx[0]
#    
#    # Obtén los puntajes de similitud para la película seleccionada
#    sim_scores = list(enumerate(cosine_sim[idx]))
#    
#    # Ordena las películas basadas en la similitud
#    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
#    
#    # Obtén los índices de las películas más similares
#    movie_indices = [i[0] for i in sim_scores[1:6]]  # 10 películas más similares
#    
#    return df['title'].iloc[movie_indices]
#
# Prueba la función de recomendación
#print(recommendacion('toy story'))

2564           toy story 2
18352    hawaiian vacation
12578          toy story 3
19124      partysaurus rex
18354            small fry
Name: title, dtype: object
